# Experiment 0 Annotator

This annotates the text with the number of times that passage has been quoted. 

In [55]:
import sys
print(sys.version)
import pandas as pd
import nltk
%matplotlib inline
import math
from ast import literal_eval
import numpy as np
import re
from matplotlib import pyplot as plt
from colour import Color
from IPython.core.display import HTML
from matplotlib import cm
from matplotlib.colors import rgb2hex
plt.rcParams["figure.figsize"] = [16, 6]

3.10.2 (v3.10.2:a58ebcc701, Jan 13 2022, 14:50:16) [Clang 13.0.0 (clang-1300.0.29.30)]


In [56]:
orig = pd.read_json('nocite.json')
orig['Decade'] = orig['publicationYear'] - (orig['publicationYear'] % 10)
df = orig[orig['Decade']==2010]
df.head()

,abstract,creator,datePublished,docSubType,docType,fullText,id,identifier,isPartOf,issueNumber,...,tdmCategory,title,url,volumeNumber,wordCount,numMatches,Locations in A,Locations in B,subTitle,Decade
10,This article investigates the practices of iti...,[Alexander Rocklin],2015-09-01,research-article,article,[Obeah and the Politics of Religion's Making a...,http://www.jstor.org/stable/24488181,"[{'name': 'issn', 'value': '00027189'}, {'name...",Journal of the American Academy of Religion,3,...,[Religion - Theology],Obeah and the Politics of Religion's Making an...,http://www.jstor.org/stable/24488181,83,11337,0,[],[],NaN,2010
12,NaN,[CHUNG-KANG KIM],2015-09-01,research-article,article,"[Nation, Subculture, and Queer Representation:...",http://www.jstor.org/stable/24616518,"[{'name': 'issn', 'value': '10434070'}, {'name...",Journal of the History of Sexuality,3,...,"[Arts - Performing arts, Philosophy - Applied ...","Nation, Subculture, and Queer Representation: ...",http://www.jstor.org/stable/24616518,24,10736,0,[],[],NaN,2010
13,NaN,[Matthew J. Lavin],2011-01-01,research-article,article,"[""The Door of Opportunity"" by Southwestern Eng...",http://www.jstor.org/stable/43025266,"[{'name': 'issn', 'value': '00433462'}, {'name...",Western American Literature,4,...,"[Arts - Literature, Philosophy - Applied philo...",Clean Hands and an Iron Face: Frontier Masculi...,http://www.jstor.org/stable/43025266,45,8406,0,[],[],NaN,2010
14,A disquieting window panel shows the crucified...,[Corine Schleif],2014-01-01,research-article,article,[The Crucifixion with Virtues in Stained Glass...,http://www.jstor.org/stable/24191440,"[{'name': 'issn', 'value': '00754250'}, {'name...",Journal of Glass Studies,NaN,...,"[Arts - Art history, Arts - Performing arts]",The Crucifixion with Virtues in Stained Glass:...,http://www.jstor.org/stable/24191440,56,12566,0,[],[],NaN,2010
18,A survey of the literature published in Englis...,[Frank Episale],2012-04-01,research-article,article,"[LITERATURE REVIEW Gender, Tradition, and Cult...",http://www.jstor.org/stable/23359546,"[{'name': 'issn', 'value': '07425457'}, {'name...",Asian Theatre Journal,1,...,"[Arts - Art history, Arts - Performing arts]","Gender, Tradition, and Culture in Translation:...",http://www.jstor.org/stable/23359546,29,9266,0,[],[],NaN,2010


In [57]:
# Adapted from text-matcher
class Text: 
    def __init__(self, filename): 
        self.filename = filename
        tokenizer = nltk.RegexpTokenizer('[a-zA-Z]\w+\'?\w*') # A custom regex tokenizer. 
        spans = list(tokenizer.span_tokenize(self.text))
        # Take note of how many spans there are in the text
        self.length = spans[-1][-1] 
        
    @property
    def text(self):
        """ Reads the file in memory. """
        f = open(self.filename, encoding='utf-8', errors='ignore')
        return f.read() 

    @property
    def tokens(self, removeStopwords=True): 
        """ Tokenizes the text, breaking it up into words, removing punctuation. """
        tokenizer = nltk.RegexpTokenizer('[a-zA-Z]\w+\'?\w*') # A custom regex tokenizer. 
        spans = list(tokenizer.span_tokenize(self.text))
        # Take note of how many spans there are in the text
        self.length = spans[-1][-1] 
        tokens = tokenizer.tokenize(self.text)
        tokens = [ token.lower() for token in tokens ] # make them lowercase
        if not removeStopwords: 
            self.spans = spans
            return tokens
        tokenSpans = list(zip(tokens, spans)) # zip it up
        stopwords = nltk.corpus.stopwords.words('english') # get stopwords
        tokenSpans = [ token for token in tokenSpans if token[0] not in stopwords ] # remove stopwords from zip
        self.spans = [ x[1] for x in tokenSpans ] # unzip; get spans
        return [ x[0] for x in tokenSpans ] # unzip; get tokens

In [58]:
mm = Text('nocite_pages.txt')

In [59]:
# Get the size of the text. 
textALength = mm.length

# I don't know why, but without the offset the novel ends too soon,
# with "unvisited tomb." This fixes it. 
offset = 2
textALength += offset
print(textALength)

# Make an empty array the size of the text. 
tally = np.zeros(textALength, dtype=np.int)

510344


/var/folders/_g/ykvg1w8n5g7f967fyjqxrpgc0000gn/T/ipykernel_32735/807644614.py:11: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  tally = np.zeros(textALength, dtype=np.int)


In [60]:
# Read the locations from the CSV file, and literally evaluate them into lists. 
locations = df['Locations in A']
#locations = locations.apply(literal_eval)

In [61]:
# Tally up every time a letter in the novel is quoted. 
for article in locations: 
    for locRange in article: 
         for i in range(locRange[0], min(locRange[1]+1, len(tally))):
                tally[i] += 1

In [62]:
# Make a color list in hex for all the values in the tally. 
# Let's hope there aren't too many. 
colors = list(np.arange(5,75,70/(tally.max()+1)))
colorList = colors

In [63]:
# Create a CSS Stylesheet for each color value in the map. 
colorCSS = ""
for i, color in zip(range(0, tally.max()+1), colorList): 
    colorCSS += ".c-%s { font-size: %spx; }\n" % (i, color)

In [64]:
n = 50

checkpoints = np.linspace(0, textALength, n).round()
checkpoints = [int(point) for point in checkpoints]

In [65]:
def span(val): 
    return '<span class="c-%s">' % val

previousVal = None
for i, valChar in enumerate(zip(tally, mm.text)):
    val, char = valChar[0], valChar[1]
    if previousVal == None: 
        # First character. 
        out = '<span class="c-%s">' % val
    elif val != previousVal: 
        out += '</span><span class="c-%s">' % val
    if i in checkpoints: 
        out += '<a name="b-%s"></a>' % checkpoints.index(i)
    out += char
    previousVal = val

In [ ]:
html = """<!DOCTYPE html>
<html>
<head>
  <link href="https://fonts.googleapis.com/css?family=Raleway" rel="stylesheet"> 
  <style>
  pre { 
      font-family: Raleway, serif; 
    }
  main { 
      width: 40em; 
      margin: 2em auto; 
  }
  %s
  </style>
  """ % (colorCSS)
html += """
  </head>
  <body><main><pre>%s</pre></main></body></html>
  """ % (out)

In [ ]:
with open('nocite_2010.html', 'w') as f: 
    f.write(html)
    f.close()